# Other Output Functions 
This notebook houses functions that create necessary outputs for forecast automation but do not fall under the 'Data Configuration' or 'Data Manipulation' umbrella. 

1. Persons Population by MGRA Function 
2. Maybe something with households

In [7]:
# Import Libraries
import pandas as pd
import numpy as np
import yaml
import os
import pyodbc
import glob
import copy
import traceback
import warnings
warnings.filterwarnings('ignore')

# Persons Population by MGRA Functions 

## Inputs

In [58]:
# TODO: Could build in the config file all of these file paths 
person_file_paths = ['T:/socioec/Current_Projects/XPEF36/abm_csv/new_parking/households_2016_05.csv',
'T:/socioec/Current_Projects/XPEF36/abm_csv/new_parking/households_2018_05.csv',
'T:/socioec/Current_Projects/XPEF36/abm_csv/new_parking/households_2020_05.csv',
'T:/socioec/Current_Projects/XPEF36/abm_csv/new_parking/households_2023_05.csv',
'T:/socioec/Current_Projects/XPEF36/abm_csv/new_parking/households_2025_05.csv',
'T:/socioec/Current_Projects/XPEF36/abm_csv/new_parking/households_2026_05.csv',
'T:/socioec/Current_Projects/XPEF36/abm_csv/new_parking/households_2029_05.csv',
'T:/socioec/Current_Projects/XPEF36/abm_csv/new_parking/households_2030_05.csv',
'T:/socioec/Current_Projects/XPEF36/abm_csv/new_parking/households_2032_05.csv',
'T:/socioec/Current_Projects/XPEF36/abm_csv/new_parking/households_2035_05.csv',
'T:/socioec/Current_Projects/XPEF36/abm_csv/new_parking/households_2040_05.csv',
'T:/socioec/Current_Projects/XPEF36/abm_csv/new_parking/households_2045_05.csv',
'T:/socioec/Current_Projects/XPEF36/abm_csv/new_parking/households_2050_05.csv',
]

output_file_path = 'J:/DataScience/DataQuality/QAQC/forecast_automation/mgra_series_13_outputs_CSV_data/other_outputs/'

In [ ]:
"T:/socioec/Current_Projects/XPEF40/abm_csv/mgra13_based_input2016_01_np.csv"

In [13]:
# TODO: Could build in the config file all of these file paths 
person_file_paths = ['T:/socioec/Current_Projects/XPEF40/abm_csv/households_2016_01.csv',
'T:/socioec/Current_Projects/XPEF40/abm_csv/households_2018_01.csv',
'T:/socioec/Current_Projects/XPEF40/abm_csv/households_2020_01.csv',
'T:/socioec/Current_Projects/XPEF40/abm_csv/households_2023_01.csv',
'T:/socioec/Current_Projects/XPEF40/abm_csv/households_2025_01.csv',
'T:/socioec/Current_Projects/XPEF40/abm_csv/households_2026_01.csv',
'T:/socioec/Current_Projects/XPEF40/abm_csv/households_2029_01.csv',
'T:/socioec/Current_Projects/XPEF40/abm_csv/households_2030_01.csv',
'T:/socioec/Current_Projects/XPEF40/abm_csv/households_2032_01.csv',
'T:/socioec/Current_Projects/XPEF40/abm_csv/households_2035_01.csv',
'T:/socioec/Current_Projects/XPEF40/abm_csv/households_2040_01.csv',
'T:/socioec/Current_Projects/XPEF40/abm_csv/households_2045_01.csv',
'T:/socioec/Current_Projects/XPEF40/abm_csv/households_2050_01.csv',
]

output_file_path = 'J:/DataScience/DataQuality/QAQC/forecast_automation/mgra_series_13_outputs_CSV_data/other_outputs/'

## Household Population Function 
Coming from the Households Dataset

In [14]:
def hh_population_from_households(DSID, person_file_paths, output_file_path, geo_level, output_to_j_drive, no_gq:False, gq_only:False):
    '''The default is an output with GQ and non-gq'''
    final_df = pd.DataFrame()
    for path in person_file_paths:
        year_from_string = path.split("/")[-1].split('_')[1]
        temp_df = pd.read_csv(path, usecols=[geo_level, 'persons', 'unittype'])

        if (no_gq & gq_only):
            print('You selected no GQ and GQ, please fix this in the input.')
            break

        if no_gq:
            temp_df = temp_df[temp_df['unittype'] == 0][[geo_level, 'persons']] # zero is non-GQ. 1 is GQ
        elif gq_only:
            temp_df = temp_df[temp_df['unittype'] == 1][[geo_level, 'persons']] # zero is non-GQ. 1 is GQ
        else: 
            temp_df = temp_df[[geo_level, 'persons']]

        temp_df['year'] = int(year_from_string)
        temp_df = temp_df.groupby(['year', geo_level]).sum().reset_index()
        temp_df.columns = ['year', geo_level, 'total_pop']
        final_df = pd.concat([final_df, temp_df])
        print(f"{year_from_string} has been updated")
        print(f"final_df length is: {len(final_df)}")
    
    # Output the data 
    if output_to_j_drive == 'Y':
        if no_gq:
            final_df.to_csv(output_file_path + f"{geo_level}_households_total_pop_non_gq_DS{DSID}_QA.csv", index=False)
            return final_df
        elif gq_only:
            final_df.to_csv(output_file_path + f"{geo_level}_households_total_pop_gq_only_DS{DSID}_QA.csv", index=False) 
        else:
            final_df.to_csv(output_file_path + f"{geo_level}_households_total_gq_and_non_gq_DS{DSID}_QA.csv", index=False)                        
    else:
        return final_df

In [15]:
total_pop_ds42_mgra = hh_population_from_households('99', person_file_paths, output_file_path, 'mgra',output_to_j_drive='Y', no_gq=False, gq_only=False)
total_pop_ds42_mgra

2016 has been updated
final_df length is: 17854
2018 has been updated
final_df length is: 35702
2020 has been updated
final_df length is: 53625
2023 has been updated
final_df length is: 71765
2025 has been updated
final_df length is: 90016
2026 has been updated
final_df length is: 108321
2029 has been updated
final_df length is: 126765
2030 has been updated
final_df length is: 145220
2032 has been updated
final_df length is: 163714
2035 has been updated
final_df length is: 182252
2040 has been updated
final_df length is: 200812
2045 has been updated
final_df length is: 219393
2050 has been updated
final_df length is: 237980


In [16]:
total_pop_ds42_mgra_non_gq = hh_population_from_households('99', person_file_paths, output_file_path, 'mgra',output_to_j_drive='Y', no_gq=True, gq_only=False)
total_pop_ds42_mgra_non_gq

2016 has been updated
final_df length is: 17831
2018 has been updated
final_df length is: 35656
2020 has been updated
final_df length is: 53556
2023 has been updated
final_df length is: 71673
2025 has been updated
final_df length is: 89902
2026 has been updated
final_df length is: 108185
2029 has been updated
final_df length is: 126607
2030 has been updated
final_df length is: 145041
2032 has been updated
final_df length is: 163514
2035 has been updated
final_df length is: 182031
2040 has been updated
final_df length is: 200571
2045 has been updated
final_df length is: 219133
2050 has been updated
final_df length is: 237701


,year,mgra,total_pop
0,2016,1,41
1,2016,2,81
2,2016,3,111
3,2016,4,73
4,2016,5,63
...,...,...,...
18563,2050,22996,236
18564,2050,22998,196
18565,2050,23000,307
18566,2050,23001,184


In [17]:
total_pop_ds42_mgra_gq_only = hh_population_from_households('99', person_file_paths, output_file_path, 'mgra',output_to_j_drive='Y', no_gq=False, gq_only=True)
total_pop_ds42_mgra_gq_only

2016 has been updated
final_df length is: 44
2018 has been updated
final_df length is: 89
2020 has been updated
final_df length is: 134
2023 has been updated
final_df length is: 179
2025 has been updated
final_df length is: 224
2026 has been updated
final_df length is: 269
2029 has been updated
final_df length is: 314
2030 has been updated
final_df length is: 359
2032 has been updated
final_df length is: 404
2035 has been updated
final_df length is: 449
2040 has been updated
final_df length is: 494
2045 has been updated
final_df length is: 539
2050 has been updated
final_df length is: 584


In [69]:
total_pop_ds42_taz = persons_dataset_population_output('42', person_file_paths, output_file_path, 'taz',output_to_j_drive='Y')

2016 has been updated
final_df length is: 4138
2018 has been updated
final_df length is: 8279
2020 has been updated
final_df length is: 12444
2023 has been updated
final_df length is: 16678
2025 has been updated
final_df length is: 20991
2026 has been updated
final_df length is: 25322
2029 has been updated
final_df length is: 29673
2030 has been updated
final_df length is: 34027
2032 has been updated
final_df length is: 38382
2035 has been updated
final_df length is: 42742
2040 has been updated
final_df length is: 47105
2045 has been updated
final_df length is: 51468
2050 has been updated
final_df length is: 55833


In [48]:
total_pop_ds42

,year,taz,total_pop
0,2016,13,2
1,2016,14,787
2,2016,15,14206
3,2016,16,4522
4,2016,17,385
...,...,...,...
4371,2050,4991,1128
4372,2050,4992,1677
4373,2050,4993,940
4374,2050,4994,507


In [70]:
# TODO: Could build in the config file all of these file paths 
person_file_paths = ['T:/socioec/Current_Projects/XPEF33/abm_csv/new_parking/households_2016_05.csv',
'T:/socioec/Current_Projects/XPEF33/abm_csv/new_parking/households_2018_05.csv',
'T:/socioec/Current_Projects/XPEF33/abm_csv/new_parking/households_2020_05.csv',
'T:/socioec/Current_Projects/XPEF33/abm_csv/new_parking/households_2023_05.csv',
'T:/socioec/Current_Projects/XPEF33/abm_csv/new_parking/households_2025_05.csv',
'T:/socioec/Current_Projects/XPEF33/abm_csv/new_parking/households_2026_05.csv',
'T:/socioec/Current_Projects/XPEF33/abm_csv/new_parking/households_2029_05.csv',
'T:/socioec/Current_Projects/XPEF33/abm_csv/new_parking/households_2030_05.csv',
'T:/socioec/Current_Projects/XPEF33/abm_csv/new_parking/households_2032_05.csv',
'T:/socioec/Current_Projects/XPEF33/abm_csv/new_parking/households_2035_05.csv',
'T:/socioec/Current_Projects/XPEF33/abm_csv/new_parking/households_2040_05.csv',
'T:/socioec/Current_Projects/XPEF33/abm_csv/new_parking/households_2045_05.csv',
'T:/socioec/Current_Projects/XPEF33/abm_csv/new_parking/households_2050_05.csv',
]

output_file_path = 'J:/DataScience/DataQuality/QAQC/forecast_automation/mgra_series_13_outputs_CSV_data/other_outputs/'

In [71]:
total_pop_ds38_mgra = persons_dataset_population_output('38', person_file_paths, output_file_path, 'mgra', output_to_j_drive='Y')

2016 has been updated
final_df length is: 17831
2018 has been updated
final_df length is: 35658
2020 has been updated
final_df length is: 53528
2023 has been updated
final_df length is: 71632
2025 has been updated
final_df length is: 89881
2026 has been updated
final_df length is: 108186
2029 has been updated
final_df length is: 126538
2030 has been updated
final_df length is: 144899
2032 has been updated
final_df length is: 163271
2035 has been updated
final_df length is: 181658
2040 has been updated
final_df length is: 200060
2045 has been updated
final_df length is: 218470
2050 has been updated
final_df length is: 236889


In [72]:
total_pop_ds38_taz = persons_dataset_population_output('38', person_file_paths, output_file_path, 'taz', output_to_j_drive='Y')

2016 has been updated
final_df length is: 4138
2018 has been updated
final_df length is: 8279
2020 has been updated
final_df length is: 12444
2023 has been updated
final_df length is: 16678
2025 has been updated
final_df length is: 20991
2026 has been updated
final_df length is: 25322
2029 has been updated
final_df length is: 29673
2030 has been updated
final_df length is: 34027
2032 has been updated
final_df length is: 38382
2035 has been updated
final_df length is: 42742
2040 has been updated
final_df length is: 47105
2045 has been updated
final_df length is: 51468
2050 has been updated
final_df length is: 55833


In [73]:
final_output= total_pop_ds38_mgra.merge(total_pop_ds42_mgra, on=['year', 'mgra'], how='inner', suffixes=['_38', '_42'])
final_output['Diff'] = final_output['total_pop_38'] - final_output['total_pop_42']

set(final_output['Diff'])

{0}

In [74]:
final_output= total_pop_ds38_taz.merge(total_pop_ds42_taz, on=['year', 'taz'], how='inner', suffixes=['_38', '_42'])
final_output['Diff'] = final_output['total_pop_38'] - final_output['total_pop_42']

set(final_output['Diff'])

{0}

In [53]:
final_output.sum()

year            113650126
taz             138602556
total_pop_38     45268536
total_pop_42     45268536
Diff                    0
dtype: int64

In [ ]:
total_pop_ds42

In [ ]:
year_from_string = 

In [8]:
'T:\socioec\Current_Projects\XPEF36\abm_csv\new_parking\households_2026_05.csv'.split("\\")[-1].split('_')[1]

'2026'

In [17]:
final_df = pd.DataFrame()
for path in person_file_paths:
    year_from_string = path.split("/")[-1].split('_')[1]
    temp_df = pd.read_csv(path, usecols=['mgra', 'persons'])
    temp_df['year'] = int(year_from_string)
    temp_df = temp_df.groupby(['year', 'mgra']).sum().reset_index()
    final_df = pd.concat([final_df, temp_df])
    print(f"{year_from_string} has been updated")
    print(f"final_df length is: {len(final_df)}")
    


2016 has been updated
final_df length is: 17854
2018 has been updated
final_df length is: 35704
2020 has been updated
final_df length is: 53597
2023 has been updated
final_df length is: 71722
2025 has been updated
final_df length is: 89992
2026 has been updated
final_df length is: 108317
2029 has been updated
final_df length is: 126689
2030 has been updated
final_df length is: 145070
2032 has been updated
final_df length is: 163462
2035 has been updated
final_df length is: 181869
2040 has been updated
final_df length is: 200291
2045 has been updated
final_df length is: 218721
2050 has been updated
final_df length is: 237160


In [18]:
final_df

,year,mgra,persons
0,2016,1,41
1,2016,2,81
2,2016,3,111
3,2016,4,73
4,2016,5,63
...,...,...,...
18434,2050,22995,109
18435,2050,22996,262
18436,2050,22998,210
18437,2050,23000,309


In [14]:
final_df

,mgra,persons,year
0,47,2,2016
1,10,2,2016
2,16,2,2016
3,27,2,2016
4,3,2,2016
...,...,...,...
1455750,6055,1,2050
1455751,6055,1,2050
1455752,6055,1,2050
1455753,6055,1,2050


In [16]:
final_df.groupby(['year', 'mgra']).sum().reset_index()

,year,mgra,persons
0,2016,1,41
1,2016,2,81
2,2016,3,111
3,2016,4,73
4,2016,5,63
...,...,...,...
237155,2050,22995,109
237156,2050,22996,262
237157,2050,22998,210
237158,2050,23000,309
